# Week 2 Exercises

In this weeks exercises you will use Numpy/Scipy to impliment some numerical algorithms and then you will use Pandas to perform a rudamentary data analysis using the KDD 98 dataset.  Along the way you will use unix/basic python from the first week as well as git to save your work.

As a first step we import the libraries we'll use later on.  This allows us to use numpy library calls by prefixing the call with np.

In [ ]:
#Import the libraries 
import numpy as np
import scipy as sp
import pandas as pd

## Matrix Manipulations
Lets first create a matrix and perform some manipulations of it.

Using numpy's matrix data structure, define the following matricies:

$$A=\left[ \begin{array}{ccc} 3 & 5 & 9 \\ 3 & 3 & 4 \\ 5 & 9 & 17 \end{array} \right]$$

$$B=\left[ \begin{array}{c} 2 \\ 1 \\ 4 \end{array} \right]$$

After this solve the matrix equation:
$$Ax = B$$

Now write three functions for matrix multiply $C=AB$ in each of the following styles:

1. By using nested for loops to impliment the naive algorithm ($C_{ij}=\sum_{k=0}^{m-1}A_{ik}B_{kj}$)
2. Using numpy's built in martrix multiplication  
3. Using Cython

The three methods should have the same answer

In [ ]:
#3 x 3 - 3 x 1
# 3 x 1 

# A = n x m = 3 x 3
# B = m x p = 3 x 1
# C = n x p = 3 x 1

def product(A,B):
    C=[[0],[0],[0]]
    C=np.empty((len(A), len(B[0])))
    
    for i in range(len(A[0])):
        for j in range(len(B[0])):
            _sum = 0
            for k in range(len(B)):
                _sum += A[i][k] * B[k][j]
            C[i][j] = _sum
    return C

A = [[3,5,9],[3,3,4],[5,9,17]]
B = [[2],[1],[4]]

product(A,B)

In [ ]:
import numpy as np
A=np.array([[3,5,9],[3,3,4],[5,9,17]])
B=np.array([[2],[1],[4]])
A.dot(B)

Now we wish to evaluate the performance of these three methods.  Write a method that given three dmiensions (a,b,c) makes a random a x b and b x c matrix and computes the product using your three functions and reports the speed of each method.

After this measure performance of each method for all $a,b,c \in \{10,100,1000,10000\}$ and plot the results.  Is one method always the fastest?  Discuss why this is or is not the case.

In [ ]:
import time

def factory(rows, cols):
    return np.random.rand(rows,cols)
    
A = factory(3,3)
B = factory(3,1)


product(A,B)

sizes = [ 10, 100, 200, 300, 400]

time_alex = []
time_np = []

for size in sizes:
    A = factory(size,size)
    B = factory(size, size)

    t0 = time.time()
    C_alex = product(A,B)
    t1 = time.time()
    elapsed = t1-t0
    time_alex.append(elapsed)
    
    t0 = time.time()
    C_np = A.dot(B)
    t1 = time.time()
    elapsed = t1-t0
    time_np.append(elapsed)
    
time_alex
time_np

import matplotlib.pyplot as plt
%matplotlib inline

plt.scatter(sizes, time_alex, color='green')
plt.scatter(sizes, time_np, color='red')

As you can see from the plot above, my naieve solution grows at a n^3 complexity rate, while the numpy implemention is much more efficient and the growth rate remains mostly constant. 

**BONUS** Now repeat the past two problems but instead of computing the matrix product, compute a matrix's [determinant](http://en.wikipedia.org/wiki/Determinant).  Measure performance for matricies of various sizes and discuss the results.  Determinant may get impractical to calculate for not too huge of matricies, so no need to goto 1000x1000 matricies.

In [ ]:
from __future__ import division
from copy import deepcopy
import math

def recursive_determinant(X):
    """
    Find the determinant in a recursive fashion.  Very inefficient
    X - Matrix object
    """
    #Must be a square matrix
    assert X.rows == X.cols
    #Must be at least 2x2
    assert X.rows > 1

    term_list = []
    #If more than 2 rows, reduce and solve in a piecewise fashion
    if X.cols>2:
        for j in xrange(0,X.cols):
            #Remove i and j columns
            new_x = deepcopy(X)
            del new_x[0]
            new_x.del_column(j)
            #Find the multiplier
            multiplier = X[0][j] * math.pow(-1,(2+j))
            #Recurse to find the determinant
            det = recursive_determinant(new_x)
            term_list.append(multiplier*det)
        return sum(term_list)
    else:
        return(X[0][0]*X[1][1] - X[0][1]*X[1][0])
    
class Matrix(object):
    """
    Represent a matrix and allow for basic matrix operations to be done.
    """
    def __init__(self, X):
        """
        X - a list of lists, ie [[1],[1]]
        """
        #Validate that the input is ok
        self.validate(X)
        self.X = X

    def validate(self, X):
        """
        Validate that a given list of lists is a proper matrix
        X - a list of lists
        """
        list_error = "X must be a list of lists corresponding to a matrix, with each sub-list being a row."
        #Input must be a list
        if not isinstance(X, list):
            raise Exception(list_error)
        #All list elements must also be lists
        for i in xrange(0,len(X)):
            if not isinstance(X[i], list):
                raise Exception(list_error)

        #All rows must have equal length
        first_row_len = len(X[0])
        for i in xrange(0,len(X)):
            if len(X[i])!=first_row_len:
                raise Exception("All rows in X must be the same length.")

    def invert(self):
        """
        Invert the matrix in place.
        """
        self.X = invert(self.X)
        return self
    
    @property
    def rows(self):
        """
        Number of rows in the matrix
        """
        return len(self.X)
    
    @property
    def cols(self):
        """
        Number of columns in the matrix
        """
        return len(self.X[0])
    
    def transpose(self):
        """
        Transpose the matrix in place.
        """
        trans = []
        for j in xrange(0,self.cols):
            row = []
            for i in xrange(0,self.rows):
                row.append(self.X[i][j])
            trans.append(row)
        self.X = trans
        return self

    def __getitem__(self, key):
        """
        Get a row of the matrix, ie m=Matrix([[1],[1]]); m[0]
        """
        return self.X[key]

    def __setitem__(self, key, value):
        """
        Set a row of the matrix, ie m=Matrix([[1],[1]]); m[0] = [2]
        """
        assert self.cols == len(value)
        self.X[key] = value

    def set_column(self, key, value):
        """
        Set a column to a specific value
        """
        assert self.rows == len(value)
        for i in xrange(0,self.rows):
            self.X[i][key] = value[i]

    def __delitem__(self, key):
        """
        Delete a row of the matrix
        """
        del self.X[key]

    def del_column(self, key):
        """
        Delete a specified column
        """
        for i in xrange(0,self.rows):
            del self.X[i][key]

    def __len__(self):
        """
        Get the length of the matrix
        """
        return self.rows

    def __rmul__(self, Z):
        """
        Right hand multiplication, ie other_matrix * matrix
        """
        #Only 2 Matrix objects can be multiplied
        assert(isinstance(Z, Matrix))

        #Number of columns in other matrix must match number of rows in this matrix
        assert Z.cols==self.rows

        product = []
        for i in xrange(0,Z.rows):
            row = []
            for j in xrange(0,self.cols):
                row.append(row_multiply(Z.X[i], [self.X[m][j] for m in xrange(0,self.rows)]))
            product.append(row)
        return Matrix(product)

    def __mul__(self, Z):
        """
        Left hand multiplication, ie matrix * other_matrix
        """
        assert(isinstance(Z, Matrix))

        assert Z.rows==self.cols

        product = []
        for i in xrange(0,self.rows):
            row = []
            for j in xrange(0,Z.cols):
                row.append(row_multiply(self.X[i], [Z[m][j] for m in xrange(0,Z.rows)]))
            product.append(row)
        return Matrix(product)

    def __str__(self):
        """
        String representation of matrix.
        """
        return str(self.X)

    def __repr__(self):
        """
        Representation of the matrix
        """
        return str(self.X)

    @property
    def determinant(self):
        return recursive_determinant(self)
    
A = [[3,5,9],[3,3,4],[5,9,17]]
A=Matrix(A)
recursive_determinant(A)




### IO Exercises

Below is a map of various datatypes in python that you have come across and their corresponding JSON equivalents.

$$Datatypes=\left[ \begin{array}{cc} JSON & Python3 \\ object & dictionary \\ array & list \\ string & string \\ integer	& integer \\ real number & float \\ true & True \\ false & False \\ null & None  \end{array} \right]$$


There are atleast two very important python datatypes missing in the above list. 
Can you find the same?  [list the two mising python datatypes in this markdown cell below]

1. Touple
2. None
3. Sets

Now We can save the above map as a dictionary with Key-value pairs 
1. create a python dictionary named dataypes, having the above map as the Key-value pairs with Python datatypes as values and JSON equivalents as keys.
2. Save it as a pickle called datatypes and gzip the same.
3. Reload this pickle, and read the file contents and output the data in the following formatted way as given in this example - "The JSON equivalent for the Python datatype Dictionary is Object". Output similarly for the rest of the key-value pairs.
4. Save this data as a JSON but using Python datatypes as keys and JSON equivalent as values this time. 

In [ ]:
Datatypes={"object": "dictionary", "array": "list", "string":"string", "integer":"integer", "realnumber":"float", "true":"True", "false":"False", "null":"None"}

In [ ]:
import pickle
import gzip
pickle.dump(Datatypes,gzip.open('picklez0.pk.gz','wb'),0)
B=pickle.load(gzip.open('picklez0.pk.gz','rb'))
B

## Pandas Data Analysis
Pandas gives us a nice set of tools to work with columnar data (similar to R's dataframe). 
To learn how to use this it makes the most sense to use a real data set.
For this assignment we'll use the KDD Cup 1998 dataset, which can be sourced from http://kdd.ics.uci.edu/databases/kddcup98/kddcup98.html .


### Acquiring Data
First we pull the README file from the dataset into this notebook via the unix "curl" command.  Remember you can hide/minimize output cells via the button on the left of the output.

In [ ]:
!curl http://kdd.ics.uci.edu/databases/kddcup98/epsilon_mirror/readme

As you can see this README describes several files which may be of use.  In particular there are two more documentation files (DOC and DIC) we should read to get an idea of the data format.  Bring these files into the notebook.

In [ ]:
!curl http://kdd.ics.uci.edu/databases/kddcup98/epsilon_mirror/cup98doc.txt

In [ ]:
!curl http://kdd.ics.uci.edu/databases/kddcup98/epsilon_mirror/cup98dic.txt

Now we wish to download the cup98lrn.zip file and unzip it into a new subdirectory called "data".  
However, since this file is pretty big we don't want to store it on github.  
Luckily git provides the [.gitignore](http://git-scm.com/docs/gitignore) file which allows us to specify files we don't want to put into our git repository.

Please do the following steps:

1. Add the directory "data" to the .gitignore file
2. Commit the new .gitignore file
3. Create a new directory "data"
4. Download http://kdd.ics.uci.edu/databases/kddcup98/epsilon_mirror/cup98lrn.zip into the data directory
5. Unzip the cup98lrn.zip (we will only be using the unzipped version, so feel free to remove the zip file)
6. Run "git status" to show that the data directory is not an untracked file (this indicates it is ignored)

**NOTE:** These steps only need to be run once, it is advised you comment all the lines out by putting a # at the start of each line after they have run.  This will save you time in the future when you have to rerun all cells/don't want to spend a few minutes downloading the data file.

In [ ]:
path="../data/cup98LRN2.csv"
file=open(path, "r")
import pandas as pd

Now perform some basic sanity checks on the data.  Using a combination of unix/basic python answer the following questions:

1. How many lines are there?  
2. Is the file character seperated or fixed width format?
3. Is there a header?  If so how many fields are in it?
4. Do all rows have the same number of fields as the header?
5. Does anyhting in 1-4 disagree with the readme file or indicate erroneous data?

In [ ]:
!wc -l ../data/cup98LRN.txt

In [ ]:
!head -n 1 ../data/cup98LRN2.csv

In [ ]:
col_names = ["ODATEDW", "OSOURCE", "TCODE"]
kdd = pd.read_csv(path, error_bad_lines=False)
kdd.describe()

Give answers to questions 1-4 in this markdown cell:

1. 
2. 
3. 
4. 

Now load the data file into a pandas data frame called "learn".  To save some time, we've loaded the data dictionary into col_types.  

Finally split learn into two data frames, learn_y: the targets (two columns described in the documentation) and learn_x: the predictors (everything but the targets)

In [ ]:
dict_file = open("dict.dat")
col_types = [ (x.split("\t")[0], x.strip().split("\t")[1]) for x in dict_file.readlines() ]

### Summarizing Data
Now that we have loaded data into the learn table, we wish to to summarize the data.  
Write a function called summary which takes a pandas data frame and prints a summary of each column containing the following:

If the column is numeric:

1. Mean
2. Standard Deviation
3. Min/Max
4. Number of missing values (NaN, Inf, NA)

If the column is non alphabetical:

1. Number of distinct values
2. Number of missing values (NaN, INF, NA, blank/all spaces)
3. The frequency of the 3 most common values and 3 least common values

Format the output to be human readable.

For example:
> Field_1  
> mean: 50  
> std_dev: 25  
> min: 0  
> max: 100  
> missing: 5
>  
> Field_2  
> distinct_values: 100  
> missing: 10  
>  
> 3 most common:  
>   the: 1000  
>   cat: 950  
>   meows: 900  
>  
> 3 least common:  
>   dogs: 5  
>   lizards: 4  
>   eggs: 1  

 ### Pandas analysis on Calit2 data 

Import data from http://archive.ics.uci.edu/ml/machine-learning-databases/event-detection/CalIt2.data using curl

This data comes from the main door of the CalIt2 building at UCI. Observations come from 2 data streams (people flow in and out of the building), over 15 weeks, 48 time slices per day (half hour count aggregates).

Attribute Information:
1. Flow ID: 7 is out flow, 9 is in flow
2. Date: MM/DD/YY
3. Time: HH:MM:SS
4. Count: Number of counts reported for the previous half hour


#### Selecting Data ####
1. Select all data for the date July 24 2005 having flow id=7. Also output the row count of results 
2. Select all rows whose count is greater than 5. Sort the result on count in descending order and output the top 10 rows

#### Apply function ####
1. For the 10 rows outputted above, use Pandas Apply function to subtract lowest value of the 10 from all of them and then output the average value of the resulting counts
2. On the entire data, use apply function to sum all counts with flow_id=9 and date is 07/24/05

#### Indexing an Selecting ####
Exlain the following

1. loc: 
2. iloc:
3. ix:
4. at:
5. iat:

Highlight the differences by providing usecases where one is more useful than the other


Write a function to take two dates as input and return all flow ids and counts in that date range having both the dates inclusive. You can use pandas to_datetime function to convert the date to pandas datetime format 

#### Grouping ####
1. Select data in the month of August 2005 having flow id=7
2. Group the data based on date and get the max count per date

#### Stacking, Unstacking ####
1. Stack the data with count and flow_id as indexes
2. Use reset_index to reset the stacked hierarchy by 1 level. The index then will just be the counts
3. Unstack the data to get back original data

#### Pandas and Matplotlib

Plot a histogram of date vs total counts for flow_id=7 and flow_id=9 for the month of July 2005